### 1. 필요한 라이브러리들을 불러옵니다. 

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time

### 2. 셀레니움 웹드라이버를 이용해서 다음 카페 모바일의 로그인 페이지를 열어줍니다. 

In [2]:
driver = webdriver.Chrome('./driver/chromedriver')
# 설치 폴더에 주의합니다. 

driver.get('https://logins.daum.net/accounts/signinform.do?url=http%3A%2F%2Fm.cafe.daum.net%2F_myCafe%3Fnull')
# 19년 5월부터 로그인 페이지 주소가 살짝 바뀌었네요. 

time.sleep(3)
# 페이지 전환시에는 적당한 시간을 줍니다. 
# 1. 과도한 크롤링 방지.
# 2. 페이지 전환이 완료되기 전에 다음 명령 실행되는 것 방지.
#    AJAX를 사용한 페이지는 페이지 전환시 딜레이가 꼭 필요한 경우도 있습니다. 

### 3. send_keys()와 click()으로 아이디와 패스워드를 자동 입력합시다.  

In [3]:
# 수동으로 웹 브라우저에 직접 입력 후 건너뛰어도 됩니다. 

#driver.find_element_by_xpath("""//*[@id="id"]""").send_keys('') # id를 입력한다. 
#driver.find_element_by_xpath("""//*[@id="inputPwd"]""").send_keys('') # 패스워드를 입력한다. 
#driver.find_element_by_xpath("""//*[@id="loginBtn"]""").click() # 입력 버튼 클릭.

time.sleep(3)

### 4. 까페 이름과 주소를 지정해 줍니다. 

In [4]:
CAFE_NAME = 'ssaumjil' 
# 까페 이름을 넣어준다. 예제는 이종격투기... 

BOARD_NAME = 'Jntr' 
# 게시판 주소의 마지막 4자리(?)를 넣어준다.
# http://m.cafe.daum.net/ssaumjil/Jntr

### 5. 게시판으로 이동

In [5]:
driver.get('http://m.cafe.daum.net/%s/%s?boardType=' % (CAFE_NAME, BOARD_NAME))
time.sleep(3)

# 이제 위에서 지정한 까페의 게시판으로 이동합니다. 
# 바로 게시물로 이동해도 됩니만.. 
# 계정정지의 압봵이 있으니... 게시판 리스트를 보고 게시물 하나만 소박하게 스크래핑하는 식으로... 

### 6. 캡쳐

In [ ]:
inp_num = input('저장할 게시물 번호 + 엔터: ')
#inp_num = '41'
num = int(inp_num) # 뒤에 DB에 정수로 저장할 거니까 미리 형변환 해줍시다. 

url = 'http://m.cafe.daum.net/%s/%s/%s' % (CAFE_NAME, BOARD_NAME, inp_num)
driver.get(url) # 게시물의 주소로 이동합니다. 

time.sleep(3)

### 페이지 소스를 BeautifulSoup(이하 BS)에게 넘기는 것을 마지막으로 셀레니움의 역할은 끝이 납니다. 

In [7]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser') 

#repr(soup)

### 제목을 찾아봅시다. 

In [14]:
subject = soup.body.find('h3', class_='tit_subject')

# BS는 class명을 이용해서 h3을 바로 찾을 수 있습니다. 
# 클래스 명은 ID와 달리 중복이 가능합니다. 
# 태그와 클래스명까지 중복되어 겹치는 경우 첫번째 노드가 검색됩니다.  
# 다행히 이 HTML 문서에서 'tit_subject' 클래스는 한번만 사용됩니다. ^^ 
# 중복을 걱정할 필요가 없네요. 
# 중복 여부는 개발자 도구의 검색 기능을 이용해 보십시오.   

In [15]:
subject = soup.body.select_one('#mArticle > div.view_subject.\#subject_area > h3')

# BS는 selector를 이용해서 노드를 찾을 수 있습니다. 
# 개발자 도구에서 copy > selector 
# select() 메소드(함수)는 해당되는 요소를 모두 찾아서 list로 리턴합니다. 

In [ ]:
repr(subject)

In [ ]:
if subject is None:
    
    # None 객체에 .get_text() 메소드를 사용하면 에러가 발생합니다.  
    # 연속 크롤링시에는 에러를 최대한 피해줘야 합니다.  
    # 그래서 if 문으로 미리 걸러주는 겁니다.  
    
    print(url, '지워진 게시물입니다.')
    # 함수로 싸 주었다면 여기서 return을 걸면 딱 좋겠죠?
    
else:
    # 지워진 게시물이 아닌 경우 이하를 계속 실행합니다. 
    subject = subject.get_text(strip=True)  

    # 작성자를 찾아봅시다. 
    
    # 다음 카페는 일시적으로 익명 전환이 가능해서. 작성자가 공란인 경우도 있습니다. 
    # 예외 처리 해줍니다. 
    
    #soup.select_one('#mArticle > div.view_subject.\#subject_area > span.txt_subject > span:nth-child(1)')
        
    if soup.body.find('span', class_='txt_subject').find('span', class_='sr_only').get_text() == '작성자':
        nick = soup.body.find('span', class_='sr_only').next_sibling
        # next_sibling 으로 옆 노드를 찾을 수 있습니다. next_sibling.next_sibling 도 해보세요. 
    else:
        nick = ''

    # 작성시간과 조회수를 찾아봅시다. 
    
    # 클래스 명이 지정되어 있습니만,  작성시간, 조회수 2군데에서 사용되었습니다. 
    # 이런 경우는 find_all()로 찾아낸 뒤, [0], [1] 하나씩 뽑아내야합니다.  
        
    num_subject = soup.body.find_all('span', class_='num_subject')
    write_time = num_subject[0].get_text()
    views = num_subject[1].get_text()

    print(num, subject, nick, write_time, views, url)

    # 본문
    
    # 본문은 친절하게 id='article' 안에 잘 들어있습니다. 
    # id는 유일하니까 바로 find로 찾으면 됩니다.
    # 본문 내 사용자가 퍼온 HTML이 있는 경우엔 유일하지 않을 때도 많죠. ㅠ,.ㅠ

    contents = soup.body.find('div', id='article').get_text('\n', strip=True)
    print(contents)

In [16]:
driver.close()
# 브라우저를 닫습니다. 